In [23]:
from bs4 import BeautifulSoup
import urllib.request
import re

import pandas as pd

In [24]:
html_page = urllib.request.urlopen('https://www.census.gov/programs-surveys/popest.html').read()
soup = BeautifulSoup(html_page, "lxml")

In [5]:
print(soup.prettify()[:1000])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <!--[if lt IE 9]><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8"  /><![endif]-->
  <!--[if gte IE 9]><meta http-equiv="X-UA-Compatible" content="IE=edge"> <![endif]-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/etc/designs/census/bootstrap.css" rel="stylesheet" type="text/css"/>
  <link href="/etc/designs/census/jquery.css" rel="stylesheet" type="text/css"/>
  <link href="/etc/designs/census/clientlibs.css" rel="stylesheet" type="text/css"/>
  <link href="/etc.clientlibs/census-core/clientlibs.css" rel="stylesheet" type="text/css"/>
  <link href="/etc.clientlibs/census/clientlibs/core-overrides.css" rel="stylesheet" type="text/css"/>
  <link href="/etc/designs/census/images/icons/android-chrome-192x192.png" rel="icon" sizes="192x192"/>
  <link href

In [13]:
file = open("my_web_links.csv", "w")
for link in soup.findAll('a', attrs={'href': re.compile("^http")}):
    soup_link = str(link['href']) + ",\n "
    print(soup_link)
    file.write(soup_link)
file.flush()
file.close()

https://www.census.gov/en.html,
 
https://www.census.gov/topics/population/age-and-sex.html,
 
https://www.census.gov/businessandeconomy,
 
https://www.census.gov/topics/education.html,
 
https://www.census.gov/topics/preparedness.html,
 
https://www.census.gov/topics/employment.html,
 
https://www.census.gov/topics/families.html,
 
https://www.census.gov/topics/population/migration.html,
 
https://www.census.gov/geo,
 
https://www.census.gov/topics/health.html,
 
https://www.census.gov/topics/population/hispanic-origin.html,
 
https://www.census.gov/topics/housing.html,
 
https://www.census.gov/topics/income-poverty.html,
 
https://www.census.gov/topics/international-trade.html,
 
https://www.census.gov/topics/population.html,
 
https://www.census.gov/topics/population/population-estimates.html,
 
https://www.census.gov/topics/public-sector.html,
 
https://www.census.gov/topics/population/race.html,
 
https://www.census.gov/topics/research.html,
 
https://www.census.gov/topics/public-

In [ ]:
#https://www.w3schools.com/python/python_regex.asp

#Split the string at every "//" character:
txt = "https://www.census.gov/library/visualizations.html/"
x = re.split("//", txt,1)[1] #Split the string only at the first occurrence
print(x)

In [22]:
txt = "https://www.census.gov/library/visualizations.html/xxxxx"
print(hash(txt))

-5460941101147978046


In [25]:
links_series = pd.Series()
type(links_series)

pandas.core.series.Series

In [123]:
links_series = pd.Series()
for link in soup.findAll('a', attrs={'href': re.compile("^http")}):
    soup_link = str(link['href'])
    soup_link = re.split("//", soup_link,1)[1]
    new_link = pd.Series(soup_link, index=[hash(soup_link)])
    try:
        links_series = links_series.append(new_link, verify_integrity=True)
    except:
        pass

file = open("my_web_links1.csv", "w")
file.write(links_series.to_csv())
file.flush()    
file.close()

## data frame to text
https://stackoverflow.com/questions/48831802/outputting-pandas-series-to-txt-file/48832218

In [114]:
try:
  print(z)
except:
  print("An exception occurred")

An exception occurred
